# Cholesky decomposition

In [1]:
import numpy as np

def cholesky_decomposition(A):
    """
    Performs the Cholesky decomposition for a positive definite matrix A.
    
    :param A: Positive definite matrix
    :return: Lower triangular matrix L such that A = L*L^T
    """
    
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        L[i, i] = np.sqrt(A[i, i] - np.dot(L[i, :i], L[i, :i]))

        L[i+1:, i] = (A[i+1:, i] - np.dot(L[i+1:, :i], L[i, :i])) / L[i, i]

    return L


A = np.array([[5, 1, 2, 1],
             [1, 4, 1, 0],
             [2, 1, 3, 1],
             [1, 0, 1, 2]], dtype= float)

L = cholesky_decomposition(A)
print("Matrix L:\n", L)
print("Checking: A = L * L^T:\n", np.dot(L, L.T) - A < 10e-09)

Matrix L:
 [[ 2.23606798  0.          0.          0.        ]
 [ 0.4472136   1.94935887  0.          0.        ]
 [ 0.89442719  0.30779351  1.4509525   0.        ]
 [ 0.4472136  -0.10259784  0.43528575  1.26491106]]
Checking: A = L * L^T:
 [[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


# Gelss

In [2]:
import numpy as np

def gelss(A, b):
    """
    Solves the least squares problem for the Ax = b system.
    
    :param A: Coefficient matrix
    :param b: Right-hand side vector
    :return: Vector x that minimizes ||Ax - b||
    """
    
    U, S, Vt = np.linalg.svd(A, full_matrices=False)
    
    S_inv = np.zeros_like(S)
    for i in range(len(S)):
        if S[i] > 1e-10:  
            S_inv[i] = 1 / S[i]
    
    A_plus = Vt.T @ np.diag(S_inv) @ U.T
    
    x = A_plus @ b
    return x

A = np.array([[3, 2, -1],
              [2, -4, 5],
              [1, 1, 1],
             [2,2,2]], dtype=float)
b = np.array([1, 2, 3, 6], dtype=float)

x = gelss(A, b)
print("Solution x:\n", x)

print("Checking:\nAx =", np.dot(A, x))
print("b =", b)

Solution x:
 [-0.11111111  1.48148148  1.62962963]
Checking:
Ax = [1. 2. 3. 6.]
b = [1. 2. 3. 6.]


# Gelsy

In [3]:
import numpy as np
from scipy.linalg import qr

def gelsy(A, b, rcond=None):
    """
    Solves the least squares problem Ax = b using QR decomposition with column pivoting.

    Parameters:
    A (ndarray): Matrix A in the equation Ax = b.
    b (ndarray): Vector b in the equation Ax = b.
    rcond (float, optional): Threshold for determining effective rank. Defaults to machine precision times max dimension of A.

    Returns:
    x (ndarray): Solution vector x minimizing ||Ax - b||.
    """
    Q, R, P = qr(A, pivoting=True)

    if rcond is None:
        rcond = np.finfo(A.dtype).eps * max(A.shape)
    rank = np.sum(np.abs(np.diag(R)) > rcond)

    R11 = R[:rank, :rank]
    Qt_b = Q.T @ b

    y = np.zeros(R.shape[1])
    y[:rank] = np.linalg.solve(R11, Qt_b[:rank])

    x = np.zeros(R.shape[1])
    x[P] = y

    return x

# Example usage
A = np.array([[3, 2, -1],
              [2, -4, 5],
              [1, 1, 1],
              [2, 2, 2]], dtype=float)
b = np.array([1, 2, 3, 6], dtype=float)

x = gelsy(A, b)
print("Solution x:\n", x)

# Verification
print("Verification:\n Ax =", np.dot(A, x))
print("b =", b)

Solution x:
 [-0.11111111  1.48148148  1.62962963]
Verification:
 Ax = [1. 2. 3. 6.]
b = [1. 2. 3. 6.]
